In [2]:
import numpy as np 
import pandas as pd
from pulsee import simulation as sim, quantum_computing as qc

<function pulsee.simulation.FID_signal(spin, h_unperturbed, dm, acquisition_time, T2=100, theta=0, phi=0, reference_frequency=0, n_points=10)>

In [ ]:
CORRELATED = True 
QUANTUM_NUMBER = 1.5 
GAMMA_2PI = 4.00 

Pulse parameters

In [ ]:
B0 = 9 
B1 = 1e-2 * B0 
GR = 11.26 